In [1]:
import sys
sys.path.append('../')

import os
import re
from datetime import datetime, timedelta

from pyspark.sql import SparkSession
from delta.pip_utils import configure_spark_with_delta_pip

from utils.string_utils import camel_to_snake, normalize_string
from engine import SupermetricsAPI
from etl import default, get_df

In [2]:
def sm_normalize_cols(s):
    s = normalize_string(s)
    s = camel_to_snake(s)
    s = s.replace(' ', '_')
    s = re.sub(r'_+', '_', s)
    s = s.strip()
    return s

In [3]:
builder = SparkSession.builder \
    .appName('test_delta_utils') \
    .config('spark.sql.warehouse.dir', 'pyspark_tables') \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config('spark.databricks.delta.retentionDurationCheck.enabled', False) \
    .config('spark.databricks.delta.schema.autoMerge.enabled', True) \
    .config('spark.databricks.delta.checkLatestSchemaOnRead', True) \
    .config("spark.log.level", "ERROR") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .config('delta.enableChangeDataFeed', True)

spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()
spark.sql("CREATE DATABASE IF NOT EXISTS supermetrics")

tb = 'supermetrics.test1'
url = os.environ['URL']
pk = ['Sheet', 'Column A']
partition = ['Sheet']
dt_start = datetime(2022, 1, 1)
dt_end = datetime(2022, 3, 1)
dt_delta = timedelta(days=15)
sm_kwargs = {'offset': 100000}
r_kwargs = {'timeout': 600}

api = SupermetricsAPI(url, **sm_kwargs)

23/04/18 01:56:54 WARN Utils: Your hostname, spiriel resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface enp3s0)
23/04/18 01:56:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ahow/main_env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ahow/.ivy2/cache
The jars for the packages stored in: /home/ahow/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a82864aa-7cdf-42e9-8a23-5b7f25806715;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 89ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   | 

23/04/18 01:56:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# df = get_df(api, spark, dt_start, dt_end, r_kwargs=r_kwargs, replace_f=[sm_normalize_cols])
# df.show()

default(
    api, spark, tb, pk, partition,
    dt_start, dt_end, dt_delta,
    r_kwargs=r_kwargs, replace_f=[sm_normalize_cols]
)

**************************************
*********** Starting SM ETL **********
**************************************
from 01/01/2022 to 15/01/2022
extracting


KeyError: 'paginate'

In [6]:
api.last_request.text

'{"meta":{"request_id":"WSgBVhSAzjXB56vAe4xo7Ron8BbyKaaK"},"error":{"code":"LICENSE_DAILY_QUERY_LIMIT_EXCEEDED","message":"LICENSE_DAILY_QUERY_LIMIT_EXCEEDED","description":"Your license limit of 100 queries per day has been exceeded for Google Sheets."}}'